# Inclined plane example

## 1. Building flat unstructured geometries

In [ ]:
import pycpt
import lavavu

import meshio
import stripy
import numpy as np
import pygmsh as pg

from scipy.interpolate import RectBivariateSpline

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

### Using pyGmsh library

Using `pyGmsh` library, which is a Python interface to `Gmsh`: a powerful mesh generation tool:

- https://github.com/nschloe/pygmsh

The resolution of the mesh is defined based on the characteristic length parameter `lcar`.

There are several different ways of defining the mesh, here we use an approach similar to what we've done previously with `stripy` and use the border coordinates to create our delaunay triangulation.

#### Bounding box

We first initialise a pyGmsh instance and define the characteristc length `lcar`:

In [ ]:
minX, maxX = 0.0, 100000.0
minY, maxY = 0.0, 100000.0

lcar = 200
geom1 = pg.built_in.Geometry()

We then define the polygon based on the box extent:

In [ ]:
poly = geom1.add_polygon([
    [ minX,  minY, 0.0],
    [ maxX,  minY, 0.0],
    [ maxX,  maxY, 0.0],
    [ minX,  maxY, 0.0]
    ],
    lcar
    )

In [ ]:
xcoords = np.arange(minX, maxX+lcar, lcar)
ycoords = np.arange(minY, maxY+lcar, lcar)

Setting uplift values

In [ ]:
U0 = 5.e-3
u = U0 * xcoords/maxX * 1.e5

nb = xcoords.shape[0]
nU = np.tile(u, nb).reshape((nb,nb))

In [ ]:
interpolateFct = RectBivariateSpline(xcoords,ycoords,nU.T)

We then generate the mesh...

In [ ]:
mesh = pg.generate_mesh(geom1)

points = mesh.points  
cells = mesh.cells 

In [ ]:
print("Unstructured Grid Geometry Characteristics:\n")

print("Number of points         nbpt: {}".format(points.shape[0]))
print("Number of faces       nbcells: {}".format(cells['triangle'].shape[0]))

Interpolation of uplift values on the unstructured grid

In [ ]:
evalU = interpolateFct.ev(points[:,0], points[:,1])

### Writing initial surface and uplift

To create the initial grids not much needs to be done. Basically we will create an array XYZ in numpy that we will write to the disk.

In [ ]:
nZ = np.zeros(points.shape[0])

We use `meshio` to create a VTK file that will then be passed to the landscape evolution model.

In [ ]:
# Creation of the X,Y,Z coordinates of the unstructured grid
verts = np.insert(points[:,:2], 2, values=nZ, axis=1)

In [ ]:
uu = evalU*1.e-5

In [ ]:
mesh = meshio.Mesh(verts, cells, {'Z':nZ,'U':uu})
meshio.write("braun.vtk", mesh)

## 2. Running the model 

All the input parameters are specified in the `input.yml` file.

### Input file

Input files for **eSCAPE** are based on [YAML](https://circleci.com/blog/what-is-yaml-a-beginner-s-guide/) syntax.

```YAML
name: Testing time steps inspired from B&W 13

domain:
    filename: ['braun.vtk','Z']
    flowdir: 1
    bc: flat

time:
    start: 0.
    end: 1.e5
    tout: 20000.
    dt: 5000.

sea:
    position: -1000.

climate:
    - start: -1.e6
      uniform: 1.0

tectonic:
   - start: -1.e6
     mapZ: ['braun.vtk','U']

sp_br:
    Kbr: 5.e-4

sp_dep:
    Ff: 1.

diffusion:
    hillslopeK: 0.
    sedimentK: 0.

output:
    dir: 'output'
    makedir: False

```

#### Parameters 

+ `domain`: definition of the unstructured grid containing the vtk grid `filename` and the associated field (here called `Z`) as well as the flow direction method to be used `flowdir` that takes an integer value between 1 (for SFD) and 12 (for Dinf) and the boundary conditions (`bc`: 'flat', 'fixed' or 'slope')

+ `time`: the simulation time parameters defined by `start`, `end`, `tout` (the output interval) and `dt` (the internal time-step).

Follows the optional forcing conditions:

+ `sea`: the sea-level declaration with the relative sea-level `position` (m) and the sea-level `curve` which is a file containing 2 columns (time and sea-level position).

+ `climatic` & `tectonic` have the same structure with a sequence of events defined by a starting time (`start`) and either a constant value (`uniform`) or a `map`.

Then the parameters for the surface processes to simulate:

+ `sp_br`: for the _stream power law_ with a unique parameter `Kbr` representing the The erodibility coefficient which is scale-dependent and its value depend on lithology and mean precipitation rate, channel width, flood frequency, channel hydraulics. It is worth noting that the coefficient _m_ and _n_ are fixed in this version and take the value 0.5 & 1 respectively. In this example we consider that all eroded sediments  are transported as fine suspension `Ff`=1 and as such will never be redeposited.

+ `diffusion`: hillslope, stream and marine diffusion coefficients. `hillslopeK` sets the _simple creep_ transport law which states that transport rate depends linearly on topographic gradient. The marine sediment are transported based on a diffusion coefficient `sedimentK`. 

Finally, you will need to specify the output folder:

+ `output`: with `dir` the directory name and the option `makedir` that gives the possible to delete any existing output folder with the same name (if set to False) or to create a new folder with the give `dir` name plus a number at the end (e.g. outputDir_1 if set to True)

### Using Jupyter notebook environment

For small models it is possible to use the notebook environment directly and run the following set of commands:

```python
import eSCAPE

# Reading input file
model = eSCAPE.LandscapeEvolutionModel(input_globe.yml)

# Running model
model.runProcesses()

# Running model
model.destroy()
```

### Using Python file

Here we will use a Python script called `run_escape.py` located in the same folder as your input file. 

```python
import argparse
import eSCAPE as sim

# Parsing command line arguments
parser = argparse.ArgumentParser(description='This is a simple entry to run eSCAPE model.',add_help=True)
parser.add_argument('-i','--input', help='Input file name (YAML file)',required=True)
parser.add_argument('-v','--verbose',help='True/false option for verbose', required=False,action="store_true",default=False)
parser.add_argument('-l','--log',help='True/false option for PETSC log', required=False,action="store_true",default=False)

args = parser.parse_args()
if args.verbose:
  print("Input file: {}".format(args.input))
  print(" Verbose is on? {}".format(args.verbose))
  print(" PETSC log is on? {}".format(args.log))

# Reading input file
model = sim.LandscapeEvolutionModel(args.input,args.verbose,args.log)

# Running model
model.runProcesses()

# Cleaning model
model.destroy()
```

This script is basically equivalent to what you will have done in the Jupyter environment but can also be ran on multiple processors using the `mpirun` command as shown below:

In [ ]:
!mpirun -np 4 python run_escape.py -i input.yml